In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
from tqdm import tqdm

## Pysentimiento

In [3]:
filePY = "/content/drive/MyDrive/Experimentos 2023-Nov/output_pysentimiento" #archivo con resultados obtenidos de Pysentimiento
dataPy = pd.read_csv(filePY)
dataPy.head()

,text,sentiment,pysentimiento
0,Quieres saber lo que los escritores de esta pe...,0,0
1,Esta fue una pelcula muy bien guionadaGran div...,1,1
2,La pelcula no fue tan grandeEl libro es mejorP...,1,1
3,He tenido que cambiar mi opinin sobre la peor ...,0,0
4,El mundo entero est cayendo presa de una enfer...,1,2


Remover los comentarios que Pysentimiento considero como neutro para solo utilizar los que son 0 (negativos) y 1 (positivos)

In [4]:
dataPy.drop(dataPy[dataPy['pysentimiento'] == 2].index, inplace=True)

In [5]:
dataPy

,text,sentiment,pysentimiento
0,Quieres saber lo que los escritores de esta pe...,0,0
1,Esta fue una pelcula muy bien guionadaGran div...,1,1
2,La pelcula no fue tan grandeEl libro es mejorP...,1,1
3,He tenido que cambiar mi opinin sobre la peor ...,0,0
5,Prncipe de Central Park es tan absolutamente ...,0,0
...,...,...,...
885,Esta es una de las mejores pelculas romnticas ...,1,1
886,Por qu esta pelcula fallaron comercialmenteTie...,1,1
888,Esta es la segunda pelcula que he visto de Ida...,1,0
889,Esta pelcula chupEl primero fue mucho mejorNad...,0,0


In [6]:
y_test = dataPy['sentiment']
y_pred = dataPy['pysentimiento']
class_names = ['negative', 'positive']

In [7]:
print(classification_report(y_test, y_pred, target_names=class_names))

              precision    recall  f1-score   support

    negative       0.86      0.93      0.89       345
    positive       0.94      0.88      0.91       426

    accuracy                           0.90       771
   macro avg       0.90      0.90      0.90       771
weighted avg       0.90      0.90      0.90       771



## Llama 2

In [8]:
import numpy as np
import pandas as pd
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
import torch
from matplotlib import rc
#
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
model_name = "4i-ai/Llama-2-7b-alpaca-es"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
!pip install bitsandbytes

In [ ]:
!pip install accelerate

In [12]:
def create_and_prepare_model():
        compute_dtype = getattr(torch, "float16")
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=compute_dtype,
            bnb_4bit_use_double_quant=True,
        )
        model = AutoModelForCausalLM.from_pretrained(
            model_name, quantization_config=bnb_config, device_map="auto"
        )
        return model


In [13]:
model = create_and_prepare_model()

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [26]:
def generate(instruction, input=None):
    #Format the prompt to look like the training data
    if input is not None:
        prompt = "### Instruction:\n"+instruction+"\n\n### Input:\n"+input+"\n\n### Response:\n"
    else :
        prompt = "### Instruction:\n"+instruction+"\n\n### Response:\n"


    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()

    generation_output = model.generate(
            input_ids=input_ids,
            generation_config=GenerationConfig(temperature=1.0, top_p=0.75, top_k=40, num_beams=10), #hyperparameters for generation
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=150, #maximum tokens generated, increase if you want longer asnwer (up to 2048 - the length of the prompt), generation "looks" slower for longer response

    )
    for seq in generation_output.sequences:
        output = tokenizer.decode(seq, skip_special_tokens=True)
        #print(output.split("### Response:")[1].strip())
        r = output.split("### Response:")[1].strip()
    return r

In [16]:
y_text_test = dataPy['text']

In [51]:
len(y_text_test)

771

In [30]:
y_text_test[0]

'Quieres saber lo que los escritores de esta pelcula consideran divertidosUn nio robot ve que sus padres robots asesinados decapitados mientras recuerdo y luego se mueven entre sus cuerpos llamando sus nombresS qu momento cmicoEsta es la peor pelcula que he pagado por ver'

In [31]:
res = generate('la oracion '+'"'+y_text_test[0]+'"'+'es un sentimiento negativo o positivo?')

In [32]:
res

'Negativo'

In [20]:
generate('la oracion '+'"'+y_text_test[0]+'"'+'es un sentimiento negativo o positivo?')

Negativo


In [22]:
generate('la oracion '+'"'+y_text_test[1]+'"'+'es un sentimiento negativo o positivo?')

Positivo


In [42]:
resultados_llama_2 = []
contador = 0
for i in tqdm(y_text_test):
  contador = contador+1
  r = generate('la oracion '+'"'+i+'"'+'es un sentimiento negativo o positivo?')
  resultados_llama_2.append(r)
  if contador == 100:
    break




 13%|█▎        | 99/771 [09:34<1:05:00,  5.80s/it]


In [106]:
resultados_llama_2

['Negativo',
 'Positivo',
 'Positivo',
 'Negativo',
 'Negativo',
 'Positivo',
 'Negativo',
 'Negativo',
 'Positivo',
 'Positivo',
 'Positivo',
 'Negativo',
 'Positivo',
 'Es un sentimiento negativo.',
 'Negativo',
 'Positivo',
 'sentimiento negativo',
 'Negativo',
 'Positivo',
 'Negativo',
 'Positivo',
 'Positivo',
 'Positivo',
 'Negativo',
 'Negativo',
 'Negativo',
 'Es un sentimiento positivo.',
 'Positivo',
 'Negativo.',
 'Positivo.',
 'Positivo',
 'Este sentimiento es negativo.',
 'La oracion "Otra mala foto de espaolEsto es muy badSolo guardo la fotografa y la msica de Jos NietoEl resto de la pelcula es lo peor que he visto en aosPaz Vega es horribleNo lo veas" es un sentimiento negativo.',
 'Un sentimiento negativo.',
 'Positivo',
 'Negativo',
 'Negativo',
 'Positivo',
 'Positivo',
 'Negativo',
 'Negativo',
 'Positivo',
 'Positivo',
 'Negativo',
 'Negativo',
 'Positivo',
 'Es un sentimiento negativo.',
 'Negativo',
 'Positivo',
 'Positivo',
 'sentimiento negativo',
 'Positivo',
 

In [107]:
def to_sentiment(sentiment):
    #sentiment = str(sentiment)
    if sentiment == 'Positivo':
        return 1
    elif sentiment == 'Negativo':
        return 0
    else:
        return 2

In [108]:
resultados_llama_2_numeros = []
for i in resultados_llama_2:
  res = to_sentiment(i)
  resultados_llama_2_numeros.append(res)


In [109]:
len(resultados_llama_2_numeros)

100

In [110]:
resultados_llama_2_numeros

[0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 2,
 0,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 2,
 2,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 2,
 1,
 0,
 1]

In [111]:
dataPy_100 = dataPy.head(100)

In [112]:
dataPy_100 = list(dataPy_100.sentiment)

In [113]:
dataPy_100

[0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1]

Primeras 100 filas que se usaron para llama, ahora hay que quitar los que llama no establecio sentimiento

In [115]:
y_pred_llama = []
y_test_real = []
for y1, y2 in zip(resultados_llama_2_numeros, dataPy_100):
  if y1 != 2:
    #print(y1)
    y_pred_llama.append(y1)
    y_test_real.append(y2)




In [116]:
len(y_pred_llama), len(y_test_real)

(82, 82)

In [117]:
class_names = ['negative', 'positive']

In [118]:
print(classification_report(y_test_real, y_pred_llama, target_names=class_names))

              precision    recall  f1-score   support

    negative       0.95      0.95      0.95        37
    positive       0.96      0.96      0.96        45

    accuracy                           0.95        82
   macro avg       0.95      0.95      0.95        82
weighted avg       0.95      0.95      0.95        82

